# **AM 207**: Homework 5

Verena Kaynig-Fittkau and Pavlos Protopapas  <br>
**Due: 11.59 P.M. Thursday April 14th, 2016**

### Note: This homework is only for one week

<div class="alert alert-info">
<strong>
Submission by Kendrick Lo (Harvard ID: 70984997) for <br>
AM 207 - Stochastic Methods for Data Analysis, Inference, and Optimization
</strong>
<br><br>
Collaborators: G. Dominedo, Wikipedia <br><br>
We also used a previous CS205 project as reference: www.spark-n-spell.com
</div>

### Instructions:

+ Upload your answers in an ipython notebook to Canvas.

+ We will provide you imports for your ipython notebook. Please do not import additional libraries.

+ Your individual submissions should use the following filenames: AM207_YOURNAME_HW5.ipynb

+ Your code should be in code cells as part of your ipython notebook. Do not use a different language (or format). 

+ **Do not just send your code. The homework solutions should be in a report style. Be sure to add comments to your code as well as markdown cells where you describe your approach and discuss your results. **

+ Please submit your notebook in an executed status, so that we can see all the results you computed. However, we will still run your code and all cells should reproduce the output when executed. 

+ If you have multiple files (e.g. you've added code files or images) create a tarball for all files in a single file and name it: AM207_YOURNAME_HW5.tar.gz or AM207_YOURNAME_HW5.zip


### Have Fun!
_ _ _ _ _

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import scipy.stats 
import pandas as pd
import pymc as pm

import re
import numpy as np

import string

import sys

# Problem 1: HMM... I Think Your Text Got Corrupted!

In this problem you should use a Hidden Markov Model to correct typos in a text without using a dictionary. Your data is in two different text files:

* `Shakespeare_correct.txt` contains the words of some sonnets from Shakespeare
* `Shakespeare_typos.txt` contains the same text, but now some of the characters are corrupted

For convenience both text files only contain lower case letters a-z and spaces. 

First build a first order HMM:
* What are the hidden states and what are the observed states?
* What should you do to generate your HMM probability matrices?
* For some of the HMM parameters, you won't have enough training data to get representative probabilities.  For example, some of your probabilites might be 0. You should address this problem by adding a small pseudocount, similar to the motif finding problem from a previous assignment. 
* Implement the Viterbi algorithm and run it on a test portion that contains errors. Show that your Viterbi implementation can improve text of length 100, 500, 1000, and 2000. Note: To do this correctly you would have to withhold the part of the text that you use for testing when you estimate the parameters for you HMM. For the sake of this homework it is ok though to report training error instead of test error. Just be aware that the correction rate you are reporting most likely is a very optimistic estimate. 
* What correction rate do you get?

**Important**: Wikipedia has a nice article on [Viterbi](https://en.wikipedia.org/wiki/Viterbi_algorithm). **Please do not use the python implementation from this article!** (The lecture notebook also has the version from Wikipedia). Using dictionaries for Viterbi is really not intuitive and using numpy is typically faster. The article has very nice pseudo code that should enable you to easily program Viterbi by yourself. Please also refrain for this problem from using any other third party implementations. 

<div class="alert alert-danger">
NOTE: As I had already completed a working solution for word-based spellchecking rather than character-based spellchecking before clarification was made on Piazza, <strong>I have left in both solutions here</strong>. In theory, one could apply the character-based correction, followed by the word-based correction on the character-level corrected test, to improve accuracy, which I have also tried. <br><br>

Contents:
<ul>
<li>1. Word-level correction
<li>2. [Character-level correction (required)](#character)
<li>3. [Character-level + Word-level correction](#wordchar)
</ul>

<br><br>
Summary of Results:
<ul>
<li>1. We achieved around 70-75% accuracy, on unseen test data, for a first order HMM based on word-level correction, and around 10% accuracy for a second order HMM based on word-level correction.
<li>2. <strong>We achieved around 90% accuracy, on unseen test data, for a first order HMM based on character-level correction, and around 95% accuracy for a second order HMM based on character-level correction.</strong>
<li>3. We achieved around 80% accuracy, on unseen test data, on one example of applying both character-level correction (second order HMM) and word-level correction (first order HMM).
</ul>
</div>

# Word-level correction solution

<div class="alert alert-info">
The hidden states are the correctly spelled words (we assume that all words in the corpus text are correctly spelled); more generally, we can treat them as the words that were intended. The observed states are the words that we actually see, which may or may not be misspelled. <br><br>

We generate our transition probability matrix by counting the number of instances in which one word in the corpus follows another; the preceding word is the hidden state from which we are transitioning, and the following word is the hidden state to which we are transitioning. <br><br>

With respect to the emissions probability matrix, we could set one up, but as noted below, instead we calculated a probability on-demand (using probabilities derived from a Poisson distribution), allowing us to forego building this matrix (there are potentially a lot more misspelled variations that we would never see, compared to the number of words in the corpus). The downside of this is that we may have to tune some of the parameters of the distribution we use, in part because we are now not normalizing probabilities over rows of an emissions probability matrix. <br><br>

With respect to the starting probabilties, we assume all words are equally likely as the first word. Note that this would not be a good assumption if we had punctuation since some words would be more likely to begin or end a sentence than others. <br><br>

Note that it is very important to have a large corpus for this type of checking. In our example, the corpus is not that long, so we should not expect very accurate results as we may not have seen enough words and/or sequences to infer corrections.
</div>

In [58]:
with open('Shakespeare_correct.txt', 'r') as f:
    corpus = f.read()

with open('Shakespeare_typos.txt', 'r') as f:
    text_to_correct = f.read()

In [59]:
corpus

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy foe to thy sweet self too cruel thou that art now the worlds fresh ornament and only herald to the gaudy spring within thine own bud buriest thy content and tender churl makst waste in niggarding pity the world or else this glutton be to eat the worlds due by the grave and thee when forty winters shall besiege thy brow and dig deep trenches in thy beautys field thy youths proud livery so gazed on now will be a totterd weed of small worth held then being asked where all thy beauty lies where all the treasure of thy lusty days to say within thine own deep sunken eyes were an alleating shame and thriftless praise how much more praise deservd thy beautys use if thou couldst answer this f

In [60]:
text_to_correct

'fron fbirest crebtuses we eesjsf iocrease uhat therfbz bebuuys rose night never eie buu as uie siper shoumd by timf eeceasf hjt tenefr iejs might bear hjs memory but thou dontractfe to thjne pwn bsight eyfs feedsu tiy mightt flane with selgsubstanuial guel mbking a famiof where abvneance lies thy self uiy foe to thz swfeu selg top csuel thou uhat ast oow the wprlds fsesi ornbmfnu and only hframd uo the gaudy sqring wjuhin thioe pwo cue buriest tiy conuent aod tendfs dhurl nakst waste jn nighardiog pjtz tif worle or elsf thjs gluttpo be to fau uie xorles dve cz the grave bnd thee when fosuy wioters shamm betiege thy brow aod eih effp trendiet in uiy bfautys fiemd tiy youths proud livfsy so gazfd on now wjll be a tpttere xefd of snall wprti hele then bejnh askfd whese aml thy ceauuy ljes where all tie trfature og thy lvsuy days to say withio thine own effp tunken eyet wese ao alleauinh thame and thriftmess praise how mudh more praise eftervd thy beautys use ig thou covmdst antwer tiis g

<div class="alert alert-info">
It looks like all the words have been forced to lower case, and there are no punctuation marks. 
</div>

In [61]:
corpus_list = corpus.split()  # splits on white space and puts words into a list
text_list = text_to_correct.split()  # splits on white space and puts words into a list
print len(corpus_list), len(text_list)

test_lengths = [len(corpus_list[i])==len(text_list[i]) for i in xrange(len(corpus_list))]
all(test_lengths)

11597 11597


True

<div class="alert alert-info">
It looks like not only are the numbers of words in each text the same, but each word is the same length in both texts. Therefore, our task is slightly simplified as we do not have to check for more complicated corruptions (e.g. deletions of letters, addition of letters, split words, etc.) <br><br>

We are going to split out a test portion of the data to obtain an accuracy measure. If we re-used training data, the results would be very misleading since it will be able to easily catch sequences that it has obviously seen before. <br><br>

We should note that training set in terms of number of words is quite small. We expect this to be particularly problematic when trying to build higher-order HMMs with words as the unit for the classes.
</div>

In [62]:
# create test sets

cutoff = 2000  # cutoff 2000 words for testing (it would be nice to have more)

corpus_test = corpus_list[-cutoff:]
corpus_list = corpus_list[:-cutoff]

text_test = text_list[-cutoff:]
text_list = text_list[:-cutoff]

print len(corpus_list), len(text_list)

9597 9597


### First-order HMM (word-level)

In [63]:
hidden_states = list(set(corpus_list))
len(hidden_states)  # i.e. unique words in the corpus

2208

<div class="alert alert-info">
We assume that any words has equal probability of being the first. This would be more complicated if we had punctuation and certain words more frequently started (or ended) sentences than others.
</div>

In [64]:
S = len(hidden_states)
start_probability = np.ones(S) * 1.0 / S 

<div class="alert alert-info">
We now count up occurrences where one word in the corpus is followed by another. From these we can get our probabilities. If we never see two words together in the corpus, this should not mean we have ZERO probability of seeing them together in real text; therefore, we add a pseudocount to each entry to avoid zeroing out our probability calculations.
</div>

In [65]:
pc = 0.1 # add a pseudocount
transition_probability = np.zeros((S, S)) + pc

for word_idx in xrange(len(corpus_list) - 1):
    i = hidden_states.index(corpus_list[word_idx])  # convert current word to row index
    j = hidden_states.index(corpus_list[word_idx + 1])  # convert next word to column index
    transition_probability[i, j] += 1

# check we have the right number of counts
assert (np.abs((np.sum(transition_probability) - pc * S * S) - (len(corpus_list) - 1))) < 0.1  # tolerance

# normalize rows to probabilities
row_sums = transition_probability.sum(axis=1, keepdims=True)
transition_probability = transition_probability * 1.0 / np.maximum(row_sums, 1) # avoid division by zero for last word

transition_probability

array([[ 0.00045086,  0.00045086,  0.00045086, ...,  0.00045086,
         0.00045086,  0.00045086],
       [ 0.00045086,  0.00045086,  0.00045086, ...,  0.00045086,
         0.00045086,  0.00045086],
       [ 0.00045086,  0.00045086,  0.00045086, ...,  0.00045086,
         0.00045086,  0.00045086],
       ..., 
       [ 0.00045086,  0.00045086,  0.00045086, ...,  0.00045086,
         0.00045086,  0.00045086],
       [ 0.00045086,  0.00045086,  0.00045086, ...,  0.00045086,
         0.00045086,  0.00045086],
       [ 0.00045086,  0.00045086,  0.00045086, ...,  0.00045086,
         0.00045086,  0.00045086]])

<div class="alert alert-info">
We could generate emission probabilities to store in a matrix, but that is probably an inefficient use of space as there will likely be many combinations of misspelled words. Instead, we can get the emission probability on demand. We model the probability using a Poisson process: <br><br>

$$
p(x_i | Z_i) = Poisson(k, lambda) \,\, \text{if word lengths are equal, otherwise 0} \\
$$

where k is the number of characters (between the expected word and the misspelling) that do not match.
</div>

In [66]:
def get_emission_probability(prevstate_idx, nextstate_data):
    
    # prevstate_idx: the index of the hidden state
    # nextstate_data: the actual string being considered
    lamb = 0.01  # rate parameter, we fiddled with this (see discussion below)
    
    reference_word = hidden_states[prevstate_idx]
    observed_word = nextstate_data
    
    if len(reference_word) == len(observed_word):
        edit_distance = np.sum([reference_word[char] != observed_word[char] for char in xrange(len(reference_word))])
        return scipy.stats.poisson.pmf(edit_distance, lamb)
    else:
        return scipy.stats.poisson.pmf(20, lamb) # assign some small probability

In [67]:
# running this commented code (instead of code above) with the Viterbi method below 
# should give the same results as shown on Wikipedia example;
# can use to test the method as modified (using arrays instead of dictionaries, working in log space, etc.)
#
# observations = ('normal', 'cold', 'dizzy')
# observed_states = ('normal', 'cold', 'dizzy')
# hidden_states = ('Healthy', 'Fever')  # hidden states
#
# start_probability = np.array([0.6, 0.4])  # probabilities ordered according to `states`
# transition_probability = np.array([[0.7, 0.3], 
#                                   [0.4, 0.6]]) # row is origin state, column is destination state
# emission_probability = np.array([[0.5, 0.4, 0.1], 
#                                 [0.1, 0.3, 0.6]])  # row is hidden state, column is observed state (observations)
#
def get_emission_probability_from_matrix(prevstate_idx, nextstate_data, observed_states, epmatrix):
    # given index of previous state and data for next step, return emission probability
    return epmatrix[prevstate_idx, observed_states.index(nextstate_data)]

#### Main Viterbi method

In [68]:
def viterbi(data, hidden_states, observed_states, start_p, trans_p, emit_p, verbose=True):
        
    T = len(data)
    S = len(hidden_states) 
    
    V = np.zeros((T, S))  # define probability matrix with dimensions (# observations, # hidden states)
    
    # populate probability matrix for time 0
    # work in logspace
    for i in xrange(S):
        if emit_p is None:
            # call function to get emission probability
            V[0, i] = np.log(start_p[i]) + np.log(get_emission_probability(i, data[0]))
        else:
            V[0, i] = np.log(start_p[i]) + np.log(get_emission_probability_from_matrix(i, data[0],
                                                                                      observed_states,
                                                                                      emit_p))
    # Run Viterbi when t > 0
    for t in range(1, T): 
        for j in xrange(S):
            prevrow = V[t-1,:]
            transprobs = np.log(trans_p[:,j])
            if emit_p is None:
                emitprob = np.log(get_emission_probability(j, data[t]))  # a number: to broadcast
            else:
                emitprob = np.log(get_emission_probability_from_matrix(j, data[t], observed_states, emit_p))
            
            vector = prevrow + transprobs + emitprob
            assert len(vector) == S
            V[t, j] = np.max(vector)  # maximum probability of reaching state j
    
    if verbose:
        print "log probabilities (time steps 0..T)"
        for i in dptable(V, T, hidden_states):
            print i

    # get optimal path
    opt = []
    for row in xrange(T):
        opt.append(hidden_states[np.argmax(V[row, ])])
    
    if verbose:
        # The highest probability
        print '\nThe steps of states are {' + ' '.join(opt) + '} with highest probability of %s' % np.exp(max(V[-1,]))
    
    return opt

def dptable(V, T, hidden_states):
    # Print a table of steps
    # yield "  ".join(("%6d" % i) for i in xrange(T))
    for y, state in enumerate(hidden_states):
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % V[row, y]) for row in xrange(T))

<div class="alert alert-info">
Let's test the algorithm on a short sequence.
</div>

In [69]:
# let's test it on a sequence of 100 words from the test set
len_test = 100  # number of words in test sequence
assert (len_test <= len(text_test)) and (len(text_test) == len(corpus_test))

begin_index = 0
observations = text_test[begin_index : begin_index + len_test]

# not using emissions matrix
emission_probability = None
observed_states = None

In [70]:
start = time.time()
# get list of proposed corrections
proposed = viterbi(observations, hidden_states, observed_states, 
                   start_probability, transition_probability, emission_probability, verbose=False)

accuracy = np.mean([proposed[i] == corpus_test[begin_index + i] for i in xrange(len_test)])
print "%i words checked in %.2f seconds, accuracy: %.4f" % (len_test, time.time() - start, accuracy) 

100 words checked in 30.08 seconds, accuracy: 0.7600


In [71]:
# corrupted text
print ' '.join(observations)

sby tis sp tit trve and to the mpst of psaise ade somfthing mose but that is in mz thouhht whose love tp you uhougi words comf iindmost holds his ranl befpre then ouhess for the breath of wosds resqecu me gor my eunb thoughts speaking jn efgect was iu uie pspud fuml taim og ijs hseat verse bound for the prizf of all too precipus zou that did mz rjqe tiouhhts in ny bsaio jnhebrsf making thejs uomb thf womb wherein thfz grew was ju his spisit by spirits uaughu to writf abowe a mprtal piudi that struck me


In [72]:
# proposed corrected text
print ' '.join(proposed)

say tis so tis true and to the must of praise are selfdoing muse but that is in me thought whose love to you though words come highmost holds his rank before then others for the breath of words respect me for my hung thoughts speaking in effect was is die proud full warm on his usest verse bound for the prize of all too precious you that did me rude thoughts in my brain rehearse making sheds womb thy womb wherein then grew was my his spirit by special taught to write adore a mortal pride that string me


In [73]:
# actual text
print ' '.join(corpus_test[:len_test])

say tis so tis true and to the most of praise add something more but that is in my thought whose love to you though words come hindmost holds his rank before then others for the breath of words respect me for my dumb thoughts speaking in effect was it the proud full sail of his great verse bound for the prize of all too precious you that did my ripe thoughts in my brain inhearse making their tomb the womb wherein they grew was it his spirit by spirits taught to write above a mortal pitch that struck me


<div class="alert alert-info">
We looked at the words that tended to resist correction. Occasionally, if a word is real word (edit distance = 0), it gets assigned a very high emission probability even though it does not make sense in the context of the local phrase. It may be worthwhile exploring different emission probability models (or further adjusting rate parameters) to give more weight to the context in which the word appears (i.e. to the transition probabilities). We note higher-order Markov modelling might also help with this.<br><br>

We also note that if we had a larger corpus, it is likely our accuracy will increase, since currently a correct word will only be identified as such if it appears in the corpus.
</div>

In [74]:
tests = [100, 500, 1000, 2000]   # number of words in test sequence

for len_test in tests:
    
    assert (len_test <= len(text_test)) and (len(text_test) == len(corpus_test))

    begin_index = 0  # we can change this if we want to start elsewhere in the test set
    observations = text_test[begin_index : begin_index + len_test]

    start = time.time()
    proposed = viterbi(observations, hidden_states, observed_states, 
                       start_probability, transition_probability, emission_probability, verbose=False)
    accuracy = np.mean([proposed[i] == corpus_test[begin_index + i] for i in xrange(len_test)])

    print "%i words checked in %.2f seconds, accuracy: %.4f" % (len_test, time.time() - start, accuracy)
    sys.stdout.flush()

100 words checked in 28.86 seconds, accuracy: 0.7600
500 words checked in 154.18 seconds, accuracy: 0.7500
1000 words checked in 287.52 seconds, accuracy: 0.7230
2000 words checked in 581.91 seconds, accuracy: 0.7145


### Second-order HMM (word-level)

Now for a second order HMM: By using a second order HMM, you should be able to get a better correction rate.
Give an intuitive explanation why a second order HMM should give better results.
Implement your second order text correction. Hint: If you think a bit about the model you won't even have to change your Viterbi implementation.
Compare your correction rates against the first order model for text length of 100 and 500, (you can do 1000 as well if your computer is fast enough).
How well would your implementation scale to HMMs of even higher order?

<div class="alert alert-info">
Intuitively, the second order model takes into account both proceeding words when determining whether a given word is correctly spelled or not. For example, the misspelled word "nou" might be reasonably corrected to "you" if it follows "shall"; however, if you knew the preceding two words were "thou shall", then "not" makes more sense than "you". The additional context may allow for a better correction rate. <br><br>

From an implementation perspective, we note that we can modify the transition matrix: instead of storing probabilities of moving from $word_{-1}$ to $word_0$, we can store probabilities of moving from $(word_{-2}, word_{-1})$ to $(word_{-1}, word_0)$. Examining "words" as pairs of words instead allows us to re-use our first-order Viterbi algorithm, but we had to modify our transition matrix to ensure that one could only transition from state (a, b) to (b, c) and not to some other state that was not linked. <br><br>

Reference: http://stackoverflow.com/questions/20509490/viterbi-algorithm-for-second-order-hmm <br><br>

We note this particular implementation could scale to HMM's of higher order by maintaining sets of words as a state. However, we would need to be more careful when defining our transition matrix. With trigrams, for example, we would need to incorporate the fact that from one three-word-combination to another three-word combination, we must pass through an additional intermediate state and track those probabilities. We could also consider employing higher-dimensional matrices, and modify our Viterbi algorithm accordingly.
</div>

In [75]:
pairs_list = [(corpus_list[i], corpus_list[i+1]) for i in xrange(len(corpus_list) - 1)]
hidden_states_2 = list(set(pairs_list))
len(hidden_states_2)  # unique pairs in the corpus

7992

<div class="alert alert-info">
We defined our HMM matrices similar to the manner above for the first order case, but tweaking the matrices to index pairs of words.
</div>

In [76]:
# redefine start probability matrix
S2 = len(hidden_states_2)
start_probability_2 = np.ones(S2) * 1.0 / S2 

# redefine transition matrix
# we do not add a pseudo count here, as we require pairs to be linked to form a 3 word sequence
transition_probability_2 = np.zeros((S2, S2))  

for word_idx in xrange(len(corpus_list) - 2):
    i = hidden_states_2.index((corpus_list[word_idx], corpus_list[word_idx + 1])) # convert current pair to row index
    j = hidden_states_2.index((corpus_list[word_idx + 1], corpus_list[word_idx + 2])) # convert next pair to column index
    transition_probability_2[i, j] += 1

assert np.sum(transition_probability_2) == len(corpus_list) - 2  # check we have the right number of counts

# normalize rows to probabilities
row_sums = transition_probability_2.sum(axis=1, keepdims=True)
transition_probability_2 = transition_probability_2 / np.maximum(row_sums, 1)  # avoid division by zero for last pair

In [77]:
# redefine how we calculate an emission probability
def get_emission_probability(prevstate_idx, nextstate_data):
    
    # prevstate_idx: the index of the hidden state
    # nextstate_data: the actual string being considered
    lamb = 0.01
    
    reference_word1, reference_word2 = hidden_states_2[prevstate_idx]
    observed_word1, observed_word2 = nextstate_data
    
    # we have to assume that the probability of each word in the pair being misspelled is independent
    # this may not be a good assumption
    p1, p2 = 0.0, 0.0
    if len(reference_word1) == len(observed_word1):
        edit_distance = np.sum([reference_word1[char] != observed_word1[char] for char in xrange(len(reference_word1))])
        p1 = scipy.stats.poisson.pmf(edit_distance, lamb)
    else:
        p1 = scipy.stats.poisson.pmf(20, lamb) # assign some small probability
        
    if len(reference_word2) == len(observed_word2):
        edit_distance = np.sum([reference_word2[char] != observed_word2[char] for char in xrange(len(reference_word2))])
        p2 = scipy.stats.poisson.pmf(edit_distance, lamb)
    else:
        p2 = scipy.stats.poisson.pmf(20, lamb) # assign some small probability  
    
    return p1 * p2  # if both words appear in sequence, this joint probability will be higher

<div class="alert alert-info">
Let's test it.
</div>

In [78]:
# let's test it on a sequence of 100 words from the test set
len_test = 100  # number of words in test sequence
assert (len_test <= len(text_test)) and (len(text_test) == len(corpus_test))

begin_index = 0
observations = text_test[begin_index : begin_index + len_test]

# not using emissions matrix
emission_probability = None
observed_states = None

# reformat data into pairs
observations_2 = [(observations[i], observations[i+1]) for i in xrange(len(observations) - 1)]

start = time.time()
# get list of proposed corrections
proposed = viterbi(observations_2, hidden_states_2, observed_states, 
                   start_probability_2, transition_probability_2, emission_probability, verbose=False)
end = time.time() - start

In [79]:
print proposed

[('see', 'his'), ('aid', 'my'), ('so', 'not'), ('not', 'from'), ('from', 'the'), ('and', 'am'), ('am', 'beloved'), ('the', 'eyes'), ('eyes', 'of'), ('of', 'men'), ('all', 'thy'), ('thy', 'gentle'), ('glory', 'live'), ('live', 'look'), ('art', 'much'), ('what', 'is'), ('is', 'your'), ('had', 'or'), ('or', 'must'), ('the', 'world'), ('world', 'doth'), ('will', 'be'), ('be', 'the'), ('the', 'praise'), ('defect', 'for'), ('being', 'mute'), ('mute', 'when'), ('deservst', 'alone'), ('alone', 'o'), ('who', 'will'), ('will', 'believe'), ('virtue', 'hath'), ('hath', 'my'), ('excuse', 'the'), ('the', 'slow'), ('the', 'better'), ('beauty', 'do'), ('of', 'young'), ('young', 'days'), ('journey', 'on'), ('on', 'the'), ('the', 'way'), ('way', 'when'), ('with', 'white'), ('own', 'selflove'), ('selflove', 'quite'), ('do', 'define'), ('define', 'as'), ('was', 'my'), ('my', 'decay'), ('the', 'peace'), ('deeds', 'then'), ('self', 'dost'), ('wail', 'thee'), ('or', 'ten'), ('ten', 'times'), ('times', 'happy

<div class="alert alert-info">
Note that some of the pairs fail to link despite our modification to the Viterbi algorithm. Put another way, it is possible that the highest probability next two-word state is one that is diassociated from the previous two-word state. This suggests that we might be getting stuck trying to find a high-probability next word in the sequence when there is no obvious choice, and there is another option with a greater likelihood of being correct. This problem might be tempered with a larger training set. <br><br>

We now reconstruct the proposed corrections into a readable form.
</div>

In [80]:
# reconstruct sequence from word pairs
def pairs_to_list(proposed, verbose=True):
    
    wordlist = []
    warned = False
    
    for i in xrange(len(proposed)):
        if i < (len(proposed) - 1):
            try:
                assert proposed[i][1] == proposed[i+1][0]  # check that words correctly link together
            except:
                if not warned:
                    if verbose:
                        print "Warning: not all pairs linked"
                    warned = True
            wordlist.append(proposed[i][0])  # take first element
        else:  # last pair
            wordlist.append(proposed[i][0]) 
            wordlist.append(proposed[i][1])

    return wordlist

proposed2 = pairs_to_list(proposed)

In [81]:
accuracy = np.mean([proposed2[i] == corpus_test[begin_index + i] for i in xrange(len_test)])
print "%i words checked in %.2f seconds, accuracy: %.4f" % (len_test, end, accuracy) 

100 words checked in 221.80 seconds, accuracy: 0.1100


<div class="alert alert-info">
It takes longer to run (because our transition matrix has about 3 times more rows and columns), and we note that the accuracy has substantially decreased for the reason explained above (mainly we need a much larger corpus). For completeness, we now test for different lengths.
</div>

In [82]:
tests = [100, 500, 1000, 2000]   # number of words in test sequence

for len_test in tests:
    
    assert (len_test <= len(text_test)) and (len(text_test) == len(corpus_test))

    begin_index = 0  # we can change this if we want to start elsewhere in the test set
    observations = text_test[begin_index : begin_index + len_test]
    observations_2 = [(observations[i], observations[i+1]) for i in xrange(len(observations) - 1)]
    
    start = time.time()
    proposed = viterbi(observations_2, hidden_states_2, observed_states, 
                       start_probability_2, transition_probability_2, emission_probability, verbose=False)
    proposed2 = pairs_to_list(proposed, verbose=False)  # convert from pairs to sequence
    accuracy = np.mean([proposed2[i] == corpus_test[begin_index + i] for i in xrange(len_test)])

    print "%i words checked in %.2f seconds, accuracy: %.4f" % (len_test, time.time() - start, accuracy)
    sys.stdout.flush()

100 words checked in 205.88 seconds, accuracy: 0.1100
500 words checked in 1037.38 seconds, accuracy: 0.1100
1000 words checked in 2078.33 seconds, accuracy: 0.0950
2000 words checked in 4723.47 seconds, accuracy: 0.0925


In [86]:
# print ' '.join(observations)

In [87]:
# print ' '.join(proposed2)

In [88]:
# print ' '.join(corpus_test[:len_test])

---

<a id='character'></a>

# Character-level correction solution

<div class="alert alert-info">
The hidden states are the characters that were intended; the observed states are the characters that we actually see, which may or may not match. <br><br>

We generate our transition probability matrix by counting the number of instances in which one character in the corpus follows another; for the first-order HMM, the preceding character is the hidden state from which we are transitioning, and the following character is the hidden state to which we are transitioning. <br><br>

With respect to the emissions probability matrix, we could set one up based on different criteria of "distance". For example, if we knew that the text was typewritten, we may assign higher probabilities of seeing a specific typo rather than an intended character depending on how far away the other characters are from the intended one on the physical layout of the keyboard. Another possibility is to assign equal probabilities that a given character could be corrupted and appear as any other character in the character set. For simplicity, we will count the number of times a letter appears in the corrupted text when a certain character was intended, and use those counts to determine our emission probabilities. <br><br>

With respect to the starting probabilties, we count the frequency of each character in the corpus and normalize those into probabilities. Note that this we would get more accurate results if we knew we were going to be testing on strings that began with a real word, since some characters would be more likely to begin a word than others.
</div>

In [2]:
with open('Shakespeare_correct.txt', 'r') as f:
    corpus = f.read()

with open('Shakespeare_typos.txt', 'r') as f:
    text_to_correct = f.read()

In [3]:
corpus

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy foe to thy sweet self too cruel thou that art now the worlds fresh ornament and only herald to the gaudy spring within thine own bud buriest thy content and tender churl makst waste in niggarding pity the world or else this glutton be to eat the worlds due by the grave and thee when forty winters shall besiege thy brow and dig deep trenches in thy beautys field thy youths proud livery so gazed on now will be a totterd weed of small worth held then being asked where all thy beauty lies where all the treasure of thy lusty days to say within thine own deep sunken eyes were an alleating shame and thriftless praise how much more praise deservd thy beautys use if thou couldst answer this f

In [4]:
text_to_correct

'fron fbirest crebtuses we eesjsf iocrease uhat therfbz bebuuys rose night never eie buu as uie siper shoumd by timf eeceasf hjt tenefr iejs might bear hjs memory but thou dontractfe to thjne pwn bsight eyfs feedsu tiy mightt flane with selgsubstanuial guel mbking a famiof where abvneance lies thy self uiy foe to thz swfeu selg top csuel thou uhat ast oow the wprlds fsesi ornbmfnu and only hframd uo the gaudy sqring wjuhin thioe pwo cue buriest tiy conuent aod tendfs dhurl nakst waste jn nighardiog pjtz tif worle or elsf thjs gluttpo be to fau uie xorles dve cz the grave bnd thee when fosuy wioters shamm betiege thy brow aod eih effp trendiet in uiy bfautys fiemd tiy youths proud livfsy so gazfd on now wjll be a tpttere xefd of snall wprti hele then bejnh askfd whese aml thy ceauuy ljes where all tie trfature og thy lvsuy days to say withio thine own effp tunken eyet wese ao alleauinh thame and thriftmess praise how mudh more praise eftervd thy beautys use ig thou covmdst antwer tiis g

<div class="alert alert-info">
It looks like all the words have been forced to lower case, and there are no punctuation marks. 
</div>

In [5]:
corpus_list = list(corpus)  # splits by character and puts into a list
text_list = list(text_to_correct)  # splits by character and puts into a list
print len(corpus_list), len(text_list)

60208 60208


<div class="alert alert-info">
We divided the text files into training and test sets. We arbitrarily set a cutoff of around 2000 characters (given the lengths of sequences we are asked to test), and since we started training at the beginning of a word, we started our test sequence at the beginning of a word.  Note, in alternative implmentations, we could employ a larger test set and/or, for example, select a random start for a test sequence.
</div>

In [6]:
# create test sets

cutoff = 2002

corpus_test = corpus_list[-cutoff:]
corpus_list = corpus_list[:-cutoff]

text_test = text_list[-cutoff:]
text_list = text_list[:-cutoff]

print len(corpus_list), len(text_list)

58206 58206


### First-order HMM (character-level)

In [7]:
hidden_states = list(set(corpus_list))
len(hidden_states)  # i.e. unique characters in the corpus

27

In [8]:
print hidden_states

[' ', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']


<div class="alert alert-info">
This was expected from a review of the text (26 alphabetic characters and the space). However, we generalize our code here to handle a different number of characters. <br><br>

With respect to starting probabilities, one approach would be to just assume that each character is likely to appear first equally. Alternatively, if we knew we were always going to start on new words, we could modify our starting probabilities to only look at characters that begin words. Another more general possibility, which we will use here, is to count the number of times each character appears in the corpus, and use those counts to determine prior probabilities.
</div>

In [9]:
S = len(hidden_states)
start_probability = np.array([corpus_list.count(char) for char in hidden_states]) 
start_probability = start_probability * 1.0 / np.sum(start_probability)
start_probability

array([ 0.19255747,  0.05504587,  0.01463767,  0.01331478,  0.10189671,
        0.03068412,  0.01542796,  0.01840016,  0.05109439,  0.05806961,
        0.0056008 ,  0.00070439,  0.02284988,  0.03451534,  0.06349861,
        0.04899839,  0.00068721,  0.01080645,  0.0546679 ,  0.04582002,
        0.02626877,  0.07990585,  0.02142391,  0.00998179,  0.02233447,
        0.00054977,  0.00025771])

<div class="alert alert-info">
Note that the `space` character (associated with the first element of the array) appears the most often. This is fine if we think that our test string could start with a space. If not, we could set tne first count to 0 and renormalize to obtain more accurate starting probabilities. <br><br>

We now count up occurrences where one word in the corpus is followed by another. From these we can get our probabilities. If we never see two words together in the corpus, this should not mean we have ZERO probability of seeing them together in real text; therefore, we add a pseudocount to each entry to avoid zeroing out our probability calculations.
</div>

In [10]:
pc = 0.1 # add a pseudocount
transition_probability = np.zeros((S, S)) + pc

for char_idx in xrange(len(corpus_list) - 1):
    i = hidden_states.index(corpus_list[char_idx])  # convert char to row index
    j = hidden_states.index(corpus_list[char_idx + 1])  # convert next char to column index
    transition_probability[i, j] += 1

# check we have the right number of counts
assert np.abs((np.sum(transition_probability) - pc * S * S) - (len(corpus_list) - 1)) < 0.1   # tolerance

# normalize rows to probabilities
row_sums = transition_probability.sum(axis=1, keepdims=True)
# avoid potential division by zero for last character
transition_probability = transition_probability * 1.0 / np.maximum(row_sums, 1)

transition_probability

array([[  8.92084534e-06,   8.52029938e-02,   2.70390822e-02,
          5.76375817e-02,   2.04376567e-02,   4.17584770e-02,
          1.92779468e-02,   4.22937278e-02,   6.12059199e-02,
          4.56836490e-02,   4.46934351e-03,   2.14992373e-03,
          6.20980044e-02,   3.97958911e-02,   5.00548632e-02,
          2.72174991e-02,   1.34704765e-03,   2.63254146e-02,
          8.70763714e-02,   1.55311917e-02,   7.94847320e-03,
          1.74679073e-01,   7.49440217e-02,   5.62905341e-03,
          2.00808229e-02,   8.92084534e-06,   9.81292987e-05],
       [  3.46462095e-02,   3.11847070e-05,   3.52699036e-02,
          9.69844388e-03,   3.11847070e-05,   3.37106683e-02,
          2.06130913e-02,   6.26812611e-03,   6.02176693e-02,
          1.59042006e-03,   2.74737269e-02,   6.54878847e-04,
          2.68500327e-02,   8.23588112e-02,   3.11847070e-05,
          1.92752674e-01,   3.11847070e-05,   1.56235382e-02,
          7.30033991e-02,   1.26329248e-01,   2.62263386e-02,
       

<div class="alert alert-info">
We will count the number of times a letter appears in the corrupted text when a certain character was intended, and use those counts to determine our emission probabilities.
</div>

In [11]:
observed_states = list(set(text_list))  
print observed_states  # check that the observed states should be the same as the hidden states

[' ', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']


In [12]:
O = len(observed_states)

emission_probability = np.zeros((S, O)) + pc  # row is hidden state, column is observed state

for char_idx in xrange(len(corpus_list)):
    i = hidden_states.index(corpus_list[char_idx])  # convert char in corpus to row index
    j = observed_states.index(text_list[char_idx])  # convert char in corrupted text to column index
    emission_probability[i, j] += 1

# check we have the right number of counts
assert np.abs((np.sum(emission_probability) - pc * S * O) - len(corpus_list)) < 0.1 # tolerance

# normalize rows to probabilities
row_sums = emission_probability.sum(axis=1, keepdims=True)
emission_probability = emission_probability * 1.0 / row_sums

print emission_probability

def get_emission_probability_from_matrix(prevstate_idx, nextstate_data, observed_states, epmatrix):
    # given index of previous state and data for next step, return emission probability
    return epmatrix[prevstate_idx, observed_states.index(nextstate_data)]

[[  9.99768079e-01   8.92004960e-06   8.92004960e-06   8.92004960e-06
    8.92004960e-06   8.92004960e-06   8.92004960e-06   8.92004960e-06
    8.92004960e-06   8.92004960e-06   8.92004960e-06   8.92004960e-06
    8.92004960e-06   8.92004960e-06   8.92004960e-06   8.92004960e-06
    8.92004960e-06   8.92004960e-06   8.92004960e-06   8.92004960e-06
    8.92004960e-06   8.92004960e-06   8.92004960e-06   8.92004960e-06
    8.92004960e-06   8.92004960e-06   8.92004960e-06]
 [  3.11847070e-05   7.99607073e-01   3.11847070e-05   1.99613310e-01
    3.11847070e-05   3.11847070e-05   3.11847070e-05   3.11847070e-05
    3.11847070e-05   3.11847070e-05   3.11847070e-05   3.11847070e-05
    3.11847070e-05   3.11847070e-05   3.11847070e-05   3.11847070e-05
    3.11847070e-05   3.11847070e-05   3.11847070e-05   3.11847070e-05
    3.11847070e-05   3.11847070e-05   3.11847070e-05   3.11847070e-05
    3.11847070e-05   3.11847070e-05   3.11847070e-05]
 [  1.17000117e-04   1.17000117e-04   7.84017784e-01

<div class="alert alert-info">
As expected, since the spaces in the texts are aligned, we expected the emission probability for a space to another space to be higher than a space to any other character (which is non-zero because of the pseudo-count).
</div>

#### Main Viterbi method

In [13]:
def viterbi(data, hidden_states, observed_states, start_p, trans_p, emit_p, verbose=True):
        
    T = len(data)
    S = len(hidden_states) 
    
    V = np.zeros((T, S))  # define probability matrix with dimensions (# observations, # hidden states)
    
    # populate probability matrix for time 0
    # work in logspace
    for i in xrange(S):
        if emit_p is None:
            # call function to get emission probability
            V[0, i] = np.log(start_p[i]) + np.log(get_emission_probability(i, data[0]))
        else:
            V[0, i] = np.log(start_p[i]) + np.log(get_emission_probability_from_matrix(i, data[0],
                                                                                      observed_states,
                                                                                      emit_p))
    # Run Viterbi when t > 0
    for t in range(1, T): 
        for j in xrange(S):
            prevrow = V[t-1,:]
            transprobs = np.log(trans_p[:,j])
            if emit_p is None:
                emitprob = np.log(get_emission_probability(j, data[t]))  
            else:
                emitprob = np.log(get_emission_probability_from_matrix(j, data[t], observed_states, emit_p))
            vector = prevrow + transprobs + emitprob  # emitprob is a number: to broadcast
            assert len(vector) == S
            V[t, j] = np.max(vector)  # maximum probability of reaching state j
    
    if verbose:
        print "log probabilities (time steps 0..T)"
        for i in dptable(V, T, hidden_states):
            print i

    # get optimal path
    opt = []
    for row in xrange(T):
        opt.append(hidden_states[np.argmax(V[row, ])])
    
    if verbose:
        # The highest probability
        print '\nThe steps of states are {' + ' '.join(opt) + '} with highest probability of %s' % np.exp(max(V[-1,]))
    
    return opt

def dptable(V, T, hidden_states):
    # Print a table of steps
    # yield "  ".join(("%6d" % i) for i in xrange(T))
    for y, state in enumerate(hidden_states):
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % V[row, y]) for row in xrange(T))

In [14]:
# let's test it on a sequence of 100 characters from the test set
len_test = 100  # number of characters in test sequence
assert (len_test <= len(text_test)) and (len(text_test) == len(corpus_test))

begin_index = 0
observations = text_test[begin_index : begin_index + len_test]

start = time.time()
# get list of proposed corrections
proposed = viterbi(observations, hidden_states, observed_states, 
                   start_probability, transition_probability, emission_probability, verbose=False)

accuracy = np.mean([proposed[i] == corpus_test[begin_index + i] for i in xrange(len_test)])
print "%i characters checked in %.2f seconds, accuracy: %.4f" % (len_test, time.time() - start, accuracy) 

100 characters checked in 0.03 seconds, accuracy: 0.8900


In [15]:
# corrupted text
print ''.join(observations)

ttplo og coth and to hit rpbbfry iad annexee uhz bseauh buu gor his thfft in psidf og all his growth


In [16]:
# proposed corrected text
print ''.join(proposed)

ttolo of coth and to hit roabery iad annexee thy bseath but gor his theet in pride of all his growsh


In [17]:
# actual text (test set)
print ''.join(corpus_test[:len_test])

stoln of both and to his robbery had annexed thy breath but for his theft in pride of all his growth


<div class="alert alert-info">
Note that since we are only looking at individual characters, and not words as a whole, the proposed words still include gibberish. <br><br>

We now do the tests of the requested length.
</div>

In [18]:
tests = [100, 500, 1000, 2000]   # number of characters in test sequence

for len_test in tests:
    
    assert (len_test <= len(text_test)) and (len(text_test) == len(corpus_test))

    begin_index = 0  # we can change this if we want to start elsewhere in the test set
    observations = text_test[begin_index : begin_index + len_test]

    start = time.time()
    proposed = viterbi(observations, hidden_states, observed_states, 
                       start_probability, transition_probability, emission_probability, verbose=False)
    accuracy = np.mean([proposed[i] == corpus_test[begin_index + i] for i in xrange(len_test)])

    print "%i characters checked in %.2f seconds, accuracy: %.4f" % (len_test, time.time() - start, accuracy)
    sys.stdout.flush()

100 characters checked in 0.03 seconds, accuracy: 0.8900
500 characters checked in 0.14 seconds, accuracy: 0.9320
1000 characters checked in 0.23 seconds, accuracy: 0.9210
2000 characters checked in 0.44 seconds, accuracy: 0.9135


<div class="alert alert-info">
Decent accuracy. Here are the 2000 character corrupted text, corrected text, and actual text sets.
</div>

In [19]:
# corrupted text
print ''.join(observations)

ttplo og coth and to hit rpbbfry iad annexee uhz bseauh buu gor his thfft in psidf og all his growth a vengeful cankfr eat him vq uo dfati more floxest i notfe yet i nonf doule tfe but sweeu or cplour iu had stoln from thee where brt thov nuse thbt thov forgettt so long to speak og uhat which givfs thee bll thy migit sqendst uhou uhy gury on some worthlett song dbrkenjng thz power to lend bbse subjects light return fprgetful muse bnd ttraight redeem jo hentle numbert tjme so idlz spent sjng to the ear tiat dpti thy layt fsteem and gjves thy pen both skjll and arhumfnt rjte restz muse my lpves swefu face svrvfz jf time hawe any wriokle gravfn thfre if aoy be a tatirf uo decay aod make times spoils detpjted everz whfrf hiwe my love fame fbttes tian time xastet life so thov preventsu his tcythe and crooled lnife o trvbnt muse what shall ce thy bmends for thy oeglfct of truuh in beauuy eyed coth truth boe beavty on ny love depends so dosu thou tpo ane therein dignifife make answer mvse wil

In [20]:
# proposed corrected text
print ''.join(proposed)

ttolo of coth and to hit roabery iad annexee thy bseath but gor his theet in pride of all his growsh a vengeeul canker eat him vp to death more flowest i notee yet i none doule tee but sweet or colour it had stoln from thee where brt thou nuse that thou forgettt so long to speak of that which ghves thee bll thy might spendst thou thy gury on some worthlett song darkening thy power to lend base subiects light return forgetetl muse bnd ttraight redeem in hentle numbert time so idly spent sing to the ear that doth thy lays frteem and gives thy pen both skill and arhument rite resty muse my loues sweeu face survey if thme hawe any wrinkle graven there if any be a tathre to decay and make thmes spoils detoited every where hive my loue fame fattes than thme wastet life so thou preventsu his tcyshe and crooled lnife o trvbnt muse what shall ce thy blends for thy oeglfct of truth in beatty eyed coth truth boe beavty on ny loue depends so dosu thou too ane therein dignifife make answer muse wil

In [21]:
# actual text (test data)
print ''.join(corpus_test[:len_test])

stoln of both and to his robbery had annexed thy breath but for his theft in pride of all his growth a vengeful canker eat him up to death more flowers i noted yet i none could see but sweet or colour it had stoln from thee where art thou muse that thou forgetst so long to speak of that which gives thee all thy might spendst thou thy fury on some worthless song darkening thy power to lend base subjects light return forgetful muse and straight redeem in gentle numbers time so idly spent sing to the ear that doth thy lays esteem and gives thy pen both skill and argument rise resty muse my loves sweet face survey if time have any wrinkle graven there if any be a satire to decay and make times spoils despised every where give my love fame faster than time wastes life so thou preventst his scythe and crooked knife o truant muse what shall be thy amends for thy neglect of truth in beauty dyed both truth and beauty on my love depends so dost thou too and therein dignified make answer muse wil

### Second-order HMM (character-level)

Now for a second order HMM: By using a second order HMM, you should be able to get a better correction rate. Give an intuitive explanation why a second order HMM should give better results. Implement your second order text correction. Hint: If you think a bit about the model you won't even have to change your Viterbi implementation. Compare your correction rates against the first order model for text length of 100 and 500, (you can do 1000 as well if your computer is fast enough). How well would your implementation scale to HMMs of even higher order?

<div class="alert alert-info">
Intuitively, the second order model takes into account two proceeding characters when determining whether a given character is correct or not. For example, "u" might be reasonably corrected to "f" if you knew it followed an "o" (i.e. "ou" -> "of"); however, if you knew the preceding two characters were "y" and "o", then leaving the "u" as-is ("you") makes more sense than changing the "u" to an "f" ("yof"). The additional context may allow for a better correction rate. <br><br>

From an implementation perspective, we note that we can modify the transition matrix: instead of storing probabilities of moving from $char_{-1}$ to $char_0$, we can store probabilities of moving from $(char_{-2}, char_{-1})$ to $(char_{-1}, char_0)$. Examining pairs of characters instead of individual ones allows us to re-use our first-order Viterbi algorithm. <br><br>

Reference: http://stackoverflow.com/questions/20509490/viterbi-algorithm-for-second-order-hmm <br><br>

We note this particular implementation could scale to HMM's of higher order by maintaining sets of characters as a state. However, we would need to be more careful when defining our transition and emission matrices. With trigrams, for example, we would need to incorporate the fact that from one three-character-combination to another three-character combination, we must pass through an additional intermediate state and track those probabilities. We could also consider employing higher-dimensional matrices, and modify our Viterbi algorithm accordingly. More importantly, any higher-order model will require a significant amount more training data, to better ensure that the probabilities calculated for different sequences will make sense.
</div>

In [22]:
# We only looked at pairs of characters that occurred in the training data, to save space
pairs_list = [(corpus_list[i], corpus_list[i+1]) for i in xrange(len(corpus_list) - 1)]
hidden_states_2 = list(set(pairs_list))
len(hidden_states_2)  # unique pairs of characters in the corpus

425

<div class="alert alert-info">
With respect to starting probabilities, we counted the number of times each pair of characters appears in the corpus (training data), and used those counts to determine prior probabilities. Higher frequencies of occurrence of the pair may suggest a higher potential to appear as the first pair of characters being tested.
</div>

In [23]:
# redefine start probability matrix
S2 = len(hidden_states_2)
start_probability_2 = np.zeros(S2) + pc

for char in xrange(len(corpus_list)-1):
    tup = (corpus_list[char], corpus_list[char+1])
    start_probability_2[hidden_states_2.index(tup)] += 1

start_probability_2 = start_probability_2 * 1.0 / np.sum(start_probability_2)

<div class="alert alert-info">
With respect to the transition matrix, we find the probability of moving from a given two character state, to a different two character state, by counting the number of times this transition occurs in the corpus (training data). We can then use those counts to determine the transition probabilities. Higher frequencies of a transition suggests that such transition is more likely to occur.
</div>

In [24]:
# redefine transition matrix
# we considered eliminating the pseudo-count here to "force" transition between certain states
# but we noted that it is still possible to correct to a more probable state (discussion below)
transition_probability_2 = np.zeros((S2, S2)) + pc

for char_idx in xrange(len(corpus_list) - 2):
    i = hidden_states_2.index((corpus_list[char_idx], corpus_list[char_idx + 1])) # convert current pair to row index
    j = hidden_states_2.index((corpus_list[char_idx + 1], corpus_list[char_idx + 2])) # convert next pair to column index
    transition_probability_2[i, j] += 1

# check we have the right number of counts
assert np.abs((np.sum(transition_probability_2) - S2 * S2 * pc) - (len(corpus_list) - 2)) < 0.1  # tolerance

# normalize rows to probabilities
row_sums = transition_probability_2.sum(axis=1, keepdims=True)
transition_probability_2 = transition_probability_2 / np.maximum(row_sums, 1)  # avoid division by zero for last pair

<div class="alert alert-info">
With respect to the emissions matrix, we build this in a manner analogous to the building of the transition matrix above; we find the probability of seeing a given two character state, when a certain two character state was intended. We do this by counting the number of times we see a certain two character state when some other two character state occurs in the corpus (training data). We can then use those counts to determine the emission probabilities. Higher frequencies suggests that a certain pair of corrupted characters is more likely to occur.
</div>

In [25]:
# redefine emissions matrix
# we need to pick up all possible observed states (alternatively we could set up for all 27 x 27 = 729 possibilities)
# but we save space this way
obs_pairs_list = [(text_list[i], text_list[i+1]) for i in xrange(len(text_list) - 1)]
obs_pairs_list2 = [(text_test[i], text_test[i+1]) for i in xrange(len(text_test) - 1)]  # we "cheat" by looking here
observed_states_2 = list(set(obs_pairs_list + obs_pairs_list2))
len(observed_states_2)  # unique pairs of characters in the corrupted text

630

In [26]:
O2 = len(observed_states_2)

emission_probability_2 = np.zeros((S2, O2)) + pc   # row is hidden state (char pair), column is observed state (char pair)

print emission_probability_2

for pair_idx in xrange(len(pairs_list)):
    i = hidden_states_2.index(pairs_list[pair_idx])  # convert char pair in corpus to row index
    j = observed_states_2.index(obs_pairs_list[pair_idx])  # convert char pair in corrupted text to column index
    emission_probability_2[i, j] += 1

# check we have the right number of counts
assert np.abs((np.sum(emission_probability_2) - pc * S2 * O2) - len(pairs_list)) < 0.1 # tolerance

# normalize rows to probabilities
row_sums = emission_probability_2.sum(axis=1, keepdims=True)
emission_probability_2 = emission_probability_2 * 1.0 / row_sums

[[ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 ..., 
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]]


In [27]:
# let's test it on a sequence of 100 characters from the test set
len_test = 100  # number of characters in test sequence
assert (len_test <= len(text_test)) and (len(text_test) == len(corpus_test))

begin_index = 0
observations = text_test[begin_index : begin_index + len_test]

# reformat data into pairs
observations_2 = [(observations[i], observations[i+1]) for i in xrange(len(observations) - 1)]

start = time.time()
# get list of proposed corrections
proposed = viterbi(observations_2, hidden_states_2, observed_states_2, 
                   start_probability_2, transition_probability_2, emission_probability_2, verbose=False)
end = time.time() - start

In [28]:
print observations_2

[('t', 't'), ('t', 'p'), ('p', 'l'), ('l', 'o'), ('o', ' '), (' ', 'o'), ('o', 'g'), ('g', ' '), (' ', 'c'), ('c', 'o'), ('o', 't'), ('t', 'h'), ('h', ' '), (' ', 'a'), ('a', 'n'), ('n', 'd'), ('d', ' '), (' ', 't'), ('t', 'o'), ('o', ' '), (' ', 'h'), ('h', 'i'), ('i', 't'), ('t', ' '), (' ', 'r'), ('r', 'p'), ('p', 'b'), ('b', 'b'), ('b', 'f'), ('f', 'r'), ('r', 'y'), ('y', ' '), (' ', 'i'), ('i', 'a'), ('a', 'd'), ('d', ' '), (' ', 'a'), ('a', 'n'), ('n', 'n'), ('n', 'e'), ('e', 'x'), ('x', 'e'), ('e', 'e'), ('e', ' '), (' ', 'u'), ('u', 'h'), ('h', 'z'), ('z', ' '), (' ', 'b'), ('b', 's'), ('s', 'e'), ('e', 'a'), ('a', 'u'), ('u', 'h'), ('h', ' '), (' ', 'b'), ('b', 'u'), ('u', 'u'), ('u', ' '), (' ', 'g'), ('g', 'o'), ('o', 'r'), ('r', ' '), (' ', 'h'), ('h', 'i'), ('i', 's'), ('s', ' '), (' ', 't'), ('t', 'h'), ('h', 'f'), ('f', 'f'), ('f', 't'), ('t', ' '), (' ', 'i'), ('i', 'n'), ('n', ' '), (' ', 'p'), ('p', 's'), ('s', 'i'), ('i', 'd'), ('d', 'f'), ('f', ' '), (' ', 'o'), ('o

In [29]:
print proposed

[('s', 't'), ('t', 'o'), ('o', 'l'), ('l', 'o'), ('o', ' '), (' ', 'o'), ('o', 'f'), ('f', ' '), (' ', 'b'), ('c', 'o'), ('o', 's'), ('t', 'h'), ('h', ' '), (' ', 'a'), ('a', 'n'), ('n', 'd'), ('d', ' '), (' ', 't'), ('t', 'o'), ('o', ' '), (' ', 'h'), ('h', 'i'), ('i', 's'), ('s', ' '), (' ', 'r'), ('r', 'o'), ('o', 'a'), ('b', 'a'), ('b', 'e'), ('e', 'r'), ('r', 'y'), ('y', ' '), (' ', 'i'), ('h', 'a'), ('a', 'd'), ('d', ' '), (' ', 'a'), ('a', 'n'), ('n', 'n'), ('n', 'e'), ('e', 'w'), ('w', 'e'), ('e', 'e'), ('e', ' '), (' ', 't'), ('t', 'h'), ('h', 'y'), ('y', ' '), (' ', 'b'), ('b', 'r'), ('r', 'e'), ('e', 'a'), ('a', 'u'), ('t', 'h'), ('h', ' '), (' ', 'b'), ('b', 'u'), ('u', 't'), ('t', ' '), (' ', 'g'), ('f', 'o'), ('o', 'r'), ('r', ' '), (' ', 'h'), ('h', 'i'), ('i', 's'), ('s', ' '), (' ', 't'), ('t', 'h'), ('h', 'e'), ('e', 'e'), ('e', 't'), ('t', ' '), (' ', 'i'), ('i', 'n'), ('n', ' '), (' ', 'p'), ('p', 'r'), ('r', 'i'), ('i', 'd'), ('d', 'e'), ('e', ' '), (' ', 'o'), ('o

In [30]:
print corpus_test[:len_test]

['s', 't', 'o', 'l', 'n', ' ', 'o', 'f', ' ', 'b', 'o', 't', 'h', ' ', 'a', 'n', 'd', ' ', 't', 'o', ' ', 'h', 'i', 's', ' ', 'r', 'o', 'b', 'b', 'e', 'r', 'y', ' ', 'h', 'a', 'd', ' ', 'a', 'n', 'n', 'e', 'x', 'e', 'd', ' ', 't', 'h', 'y', ' ', 'b', 'r', 'e', 'a', 't', 'h', ' ', 'b', 'u', 't', ' ', 'f', 'o', 'r', ' ', 'h', 'i', 's', ' ', 't', 'h', 'e', 'f', 't', ' ', 'i', 'n', ' ', 'p', 'r', 'i', 'd', 'e', ' ', 'o', 'f', ' ', 'a', 'l', 'l', ' ', 'h', 'i', 's', ' ', 'g', 'r', 'o', 'w', 't', 'h']


<div class="alert alert-info">
The depiction of the state transitions for the observed data and the proposed corrections is not helpful. So we reconstructed these paired states into readable form. We built back the proposed string by taking the first letter of each state pair, and both letters from the last pair. (Alternatively, we could choose both letters from the first pair and the second letter of each state pair for comparable results).
</div>

In [31]:
# reconstruct sequence from character pairs
def pairs_to_list(prop, verbose=True):
    
    charstring = []
    warned = False
    
    for i in xrange(len(prop)):
        if i < (len(prop) - 1):
            try:
                assert prop[i][1] == prop[i+1][0]  # check that words correctly link together
            except:
                if not warned:
                    if verbose:
                        print "Warning: not all pairs linked"
                    warned = True
            charstring.append(prop[i][0])  # take first element
        else:  # last pair
            charstring.append(prop[i][0]) 
            charstring.append(prop[i][1])

    return charstring

proposed2 = pairs_to_list(proposed)

In [32]:
accuracy = np.mean([proposed2[i] == corpus_test[begin_index + i] for i in xrange(len_test)])
print "%i characters checked in %.2f seconds, accuracy: %.4f" % (len_test, end, accuracy) 

100 characters checked in 1.10 seconds, accuracy: 0.9500


<div class="alert alert-info">
A slight increase in accuracy, at the expense of a longer run time.
</div>

In [33]:
# Let's see why we got a warning
print proposed

[('s', 't'), ('t', 'o'), ('o', 'l'), ('l', 'o'), ('o', ' '), (' ', 'o'), ('o', 'f'), ('f', ' '), (' ', 'b'), ('c', 'o'), ('o', 's'), ('t', 'h'), ('h', ' '), (' ', 'a'), ('a', 'n'), ('n', 'd'), ('d', ' '), (' ', 't'), ('t', 'o'), ('o', ' '), (' ', 'h'), ('h', 'i'), ('i', 's'), ('s', ' '), (' ', 'r'), ('r', 'o'), ('o', 'a'), ('b', 'a'), ('b', 'e'), ('e', 'r'), ('r', 'y'), ('y', ' '), (' ', 'i'), ('h', 'a'), ('a', 'd'), ('d', ' '), (' ', 'a'), ('a', 'n'), ('n', 'n'), ('n', 'e'), ('e', 'w'), ('w', 'e'), ('e', 'e'), ('e', ' '), (' ', 't'), ('t', 'h'), ('h', 'y'), ('y', ' '), (' ', 'b'), ('b', 'r'), ('r', 'e'), ('e', 'a'), ('a', 'u'), ('t', 'h'), ('h', ' '), (' ', 'b'), ('b', 'u'), ('u', 't'), ('t', ' '), (' ', 'g'), ('f', 'o'), ('o', 'r'), ('r', ' '), (' ', 'h'), ('h', 'i'), ('i', 's'), ('s', ' '), (' ', 't'), ('t', 'h'), ('h', 'e'), ('e', 'e'), ('e', 't'), ('t', ' '), (' ', 'i'), ('i', 'n'), ('n', ' '), (' ', 'p'), ('p', 'r'), ('r', 'i'), ('i', 'd'), ('d', 'e'), ('e', ' '), (' ', 'o'), ('o

<div class="alert alert-info">
Interestingly, when we take a closer look at the state transitions, we note that it is not always the case that a successive state will share the same first letter as the last letter of the previous state (see e.g. (' ', 'b'), ('c', 'o')), regardless of whether we add a pseudocount to the transition probabilities or not. This does not occur that often, but it does occur some of the time. By picking the path associated with the largest probability at each step, there may be a point in which the algorithm corrects to a new more probable state, rather than being constrained by a previously corrected state (which could still contain a typo, and may get itself stuck trying to find a next character in the sequence where there is no obvious good choice). <br><br>

We now run our algorithm on the different length test sequences:
</div>

In [34]:
tests = [100, 500, 1000, 2000]   # number of characters in test sequence

for len_test in tests:
    
    assert (len_test <= len(text_test)) and (len(text_test) == len(corpus_test))

    begin_index = 0  # we can change this if we want to start elsewhere in the test set
    observations = text_test[begin_index : begin_index + len_test]
    observations_2 = [(observations[i], observations[i+1]) for i in xrange(len(observations) - 1)]
    
    start = time.time()
    proposed = viterbi(observations_2, hidden_states_2, observed_states_2, 
                       start_probability_2, transition_probability_2, emission_probability_2, verbose=False)
    proposed2 = pairs_to_list(proposed, verbose=False)  # convert from pairs to sequence
    accuracy = np.mean([proposed2[i] == corpus_test[begin_index + i] for i in xrange(len_test)])

    print "%i characters checked in %.2f seconds, accuracy: %.4f" % (len_test, time.time() - start, accuracy)
    sys.stdout.flush()

100 characters checked in 1.09 seconds, accuracy: 0.9500
500 characters checked in 5.50 seconds, accuracy: 0.9600
1000 characters checked in 10.85 seconds, accuracy: 0.9640
2000 characters checked in 22.04 seconds, accuracy: 0.9575


<div class="alert alert-info">
Comparing to the earlier results for the first-order character level HMM:
    
<code>
100 characters checked in 0.02 seconds, accuracy: 0.8900
500 characters checked in 0.15 seconds, accuracy: 0.9320
1000 characters checked in 0.23 seconds, accuracy: 0.9210
2000 characters checked in 0.43 seconds, accuracy: 0.9135
</code>
<br>
We notice that we have experienced an overall increase in accuracy with the second-order HMM, despite having to effectively revert to a different selection if a certain state has no high-probability choice that maintains a common linking letter. More precise results might be obtained with more training data.
</div>

In [35]:
# corrupted text for length 2000
print ''.join(observations)

ttplo og coth and to hit rpbbfry iad annexee uhz bseauh buu gor his thfft in psidf og all his growth a vengeful cankfr eat him vq uo dfati more floxest i notfe yet i nonf doule tfe but sweeu or cplour iu had stoln from thee where brt thov nuse thbt thov forgettt so long to speak og uhat which givfs thee bll thy migit sqendst uhou uhy gury on some worthlett song dbrkenjng thz power to lend bbse subjects light return fprgetful muse bnd ttraight redeem jo hentle numbert tjme so idlz spent sjng to the ear tiat dpti thy layt fsteem and gjves thy pen both skjll and arhumfnt rjte restz muse my lpves swefu face svrvfz jf time hawe any wriokle gravfn thfre if aoy be a tatirf uo decay aod make times spoils detpjted everz whfrf hiwe my love fame fbttes tian time xastet life so thov preventsu his tcythe and crooled lnife o trvbnt muse what shall ce thy bmends for thy oeglfct of truuh in beauuy eyed coth truth boe beavty on ny love depends so dosu thou tpo ane therein dignifife make answer mvse wil

In [36]:
# proposed corrected text
print ''.join(proposed2)

stolo of coth and to his robbery had annewee thy breath but for his theet in pride of all his growth a vengeful canker eat him up to death more flowest i notee yet i none dould tee but sweet or colour it had stoln from thee where brt thou nuse that thou forgetst so long to speak of that which gives thee bll thy might spendst thou thy gury on some worthlest song darkening thy power to lend base subjects light resurn forgeteul muse and straight receem in hentle numbert time so idly spent sing to the ear that doth thy lays fsteem and gives thy pen both skill and argument rite resty muse my loves sweet face survey if time have any wrinkle graven there if any be a tatire to decay and make times spoils despited every where hive my love fame fattes than time wastet life so thou preventst his scythe and crooked knife o truant muse what shall be thy blends for thy oeglect of truth in beauty eyed coth truth bnd beauty on my love depends so dost thou too and therein dignifife make answer muse wil

In [37]:
# actual text (test data)
print ''.join(corpus_test[:len_test])

stoln of both and to his robbery had annexed thy breath but for his theft in pride of all his growth a vengeful canker eat him up to death more flowers i noted yet i none could see but sweet or colour it had stoln from thee where art thou muse that thou forgetst so long to speak of that which gives thee all thy might spendst thou thy fury on some worthless song darkening thy power to lend base subjects light return forgetful muse and straight redeem in gentle numbers time so idly spent sing to the ear that doth thy lays esteem and gives thy pen both skill and argument rise resty muse my loves sweet face survey if time have any wrinkle graven there if any be a satire to decay and make times spoils despised every where give my love fame faster than time wastes life so thou preventst his scythe and crooked knife o truant muse what shall be thy amends for thy neglect of truth in beauty dyed both truth and beauty on my love depends so dost thou too and therein dignified make answer muse wil

---

<a id='wordchar'></a>

# Combining character-level correction and word-level correction.

<div class="alert alert-info">
Finally, as mentioned earlier, we may also attempt to increase our accuracy by combining the two approaches we described; for example, applying the second-order character-level algorithm to the data to be corrected first (to correct character mistakes), and then applying the first-order word-level algorithm (to correct word mistakes). This may also make the proposed text more sensible as it is less likely to contain gibberish. 
</div>

In [53]:
# we repeat the word-level code here in case the first part of the notebook was not run

new_test = ''.join(proposed2).split()
new_ref = ''.join(corpus_test[:2000]).split()

print len(new_test)
print len(new_ref)

corpus_list = corpus.split()  # splits on white space and puts words into a list
text_list = text_to_correct.split()  # splits on white space and puts words into a list

# create test sets
cutoff = 2002  # cutoff ~2000 words for testing (it would be nice to have more)

corpus_test2 = new_ref # *** replaced these with results from character-level correction
corpus_list = corpus_list[:-cutoff]

text_test2 = new_test # *** replaced these with results from character-level correction
text_list = text_list[:-cutoff]

hidden_states = list(set(corpus_list))
S = len(hidden_states)
start_probability = np.ones(S) * 1.0 / S 

pc = 0.1 # add a pseudocount
transition_probability = np.zeros((S, S)) + pc

for word_idx in xrange(len(corpus_list) - 1):
    i = hidden_states.index(corpus_list[word_idx])  # convert current word to row index
    j = hidden_states.index(corpus_list[word_idx + 1])  # convert next word to column index
    transition_probability[i, j] += 1

# check we have the right number of counts
assert (np.abs((np.sum(transition_probability) - pc * S * S) - (len(corpus_list) - 1))) < 0.1   # tolerance

# normalize rows to probabilities
row_sums = transition_probability.sum(axis=1, keepdims=True)
transition_probability = transition_probability * 1.0 / np.maximum(row_sums, 1) # avoid division by zero for last word

def get_emission_probability(prevstate_idx, nextstate_data):
    
    # prevstate_idx: the index of the hidden state
    # nextstate_data: the actual string being considered
    lamb = 0.01  # rate parameter
    
    reference_word = hidden_states[prevstate_idx]
    observed_word = nextstate_data
    
    if len(reference_word) == len(observed_word):
        edit_distance = np.sum([reference_word[char] != observed_word[char] for char in xrange(len(reference_word))])
        return scipy.stats.poisson.pmf(edit_distance, lamb)
    else:
        return scipy.stats.poisson.pmf(20, lamb) # assign some small probability

389
389


In [54]:
len_test = len(new_test)  # number of words in test sequence for comparison

begin_index = 0  # we can change this if we want to start elsewhere in the test set
observations = text_test2[begin_index : begin_index + len_test]

start = time.time()
proposed = viterbi(observations, hidden_states, None, 
                   start_probability, transition_probability, None, verbose=False)

accuracy = np.mean([proposed[i] == corpus_test2[begin_index + i] for i in xrange(len_test)])

print "%i words checked in %.2f seconds, accuracy: %.4f" % (len_test, time.time() - start, accuracy)
sys.stdout.flush()

389 words checked in 112.72 seconds, accuracy: 0.8123


In [55]:
print ' '.join(observations)

stolo of coth and to his robbery had annewee thy breath but for his theet in pride of all his growth a vengeful canker eat him up to death more flowest i notee yet i none dould tee but sweet or colour it had stoln from thee where brt thou nuse that thou forgetst so long to speak of that which gives thee bll thy might spendst thou thy gury on some worthlest song darkening thy power to lend base subjects light resurn forgeteul muse and straight receem in hentle numbert time so idly spent sing to the ear that doth thy lays fsteem and gives thy pen both skill and argument rite resty muse my loves sweet face survey if time have any wrinkle graven there if any be a tatire to decay and make times spoils despited every where hive my love fame fattes than time wastet life so thou preventst his scythe and crooked knife o truant muse what shall be thy blends for thy oeglect of truth in beauty eyed coth truth bnd beauty on my love depends so dost thou too and therein dignifife make answer muse wil

In [56]:
print ' '.join(proposed)

stoln of doth and to his robbery had another thy breath but for his cheek in pride of all his growst a wasteful canker eat him up to death more flowers i noted yet i none would the but sweet or honour it had stoln from thee where art thou muse that thou farthest so long to speak of that which gives thee ill thy might special thou thy wary on some worthless song battering thy power to lend base subjects light return forgotten muse and straight review in gentle numbers time so idle spent sing to the ear that doth thy days father and gives thy pen both skill and argument rite lusty muse my loves sweet face survey if time have any wracked graves there if any be a nature to decay and make times broils despised every where hide my love fame matter than time wastes life so thou presentst his scythe and crooked knife o truest muse what shall be thy blanks for thy respect of truth in beauty eyes doth truth and beauty on my love defence so dost thou too and therein dignifies make answer muse wil

In [57]:
print ' '.join(corpus_test2[:len_test])

stoln of both and to his robbery had annexed thy breath but for his theft in pride of all his growth a vengeful canker eat him up to death more flowers i noted yet i none could see but sweet or colour it had stoln from thee where art thou muse that thou forgetst so long to speak of that which gives thee all thy might spendst thou thy fury on some worthless song darkening thy power to lend base subjects light return forgetful muse and straight redeem in gentle numbers time so idly spent sing to the ear that doth thy lays esteem and gives thy pen both skill and argument rise resty muse my loves sweet face survey if time have any wrinkle graven there if any be a satire to decay and make times spoils despised every where give my love fame faster than time wastes life so thou preventst his scythe and crooked knife o truant muse what shall be thy amends for thy neglect of truth in beauty dyed both truth and beauty on my love depends so dost thou too and therein dignified make answer muse wil

<div class="alert alert-info">
Interestingly, we did not get the improvement we expected. On a positive note, it appears that the proposed corrected text is now more readable, and contains only words that can be found in the corpus. However, it does worse than character-level correction (although comparable to, in fact slightly better than, the results when using only word-level correction). This suggests that the corruption is not likely a random process; there may be some pattern to the corruption that is not reflected in the transition and emission matrices. However, we would expect better results with a larger training set. Moreover, if we applied a second-order word level algorithm (which requires a larger corpus), we would expected better results since we would be taking into account the context of a word within a sentence.
</div>

# Extra Problem 2: Final Project Review
    
You will be contacted shortly by a TF to meet and discuss your final project proposal. Be sure to take advantage of this feedback option. Review meetings should be scheduled within the week from April 11-15. 